In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv("polynomial_regression_train.csv")
data.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Target
0,1,-0.299150,0.980381,-1.244780,0.420551,-0.352235,8.168743e-10
1,2,-0.374319,-1.179867,0.615588,0.054701,0.687182,-5.081225e-10
2,3,-1.240429,-0.139086,-0.001653,0.487685,-0.242808,4.560170e-10
3,4,-0.429881,-1.724246,-0.098046,0.896504,0.318585,-2.647546e-09
4,5,-1.726345,1.945823,1.275979,-0.474670,1.050494,-1.704606e-08


In [3]:
x = data.iloc[:,1:6]
print(x.head())
#x = (x-x.mean())/(x.std())
x = x.values
y = data.iloc[:,6]
print(y.head())
y_train = y.values

   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5
0  -0.299150   0.980381  -1.244780   0.420551  -0.352235
1  -0.374319  -1.179867   0.615588   0.054701   0.687182
2  -1.240429  -0.139086  -0.001653   0.487685  -0.242808
3  -0.429881  -1.724246  -0.098046   0.896504   0.318585
4  -1.726345   1.945823   1.275979  -0.474670   1.050494
0    8.168743e-10
1   -5.081225e-10
2    4.560170e-10
3   -2.647546e-09
4   -1.704606e-08
Name: Target, dtype: float64


In [4]:

def polynomial_features(X, degree):  # X is a numpy array 
   
    n_samples, n_features = X.shape # extracting....
    poly_features = [np.ones(n_samples)]  # Start with a bias term (x^0) # column 1 will be of ones ...

    def recursive_terms(current_degree, current_combination):
       
        if current_degree == 0:
            
            new_feature = np.prod([X[:, idx] for idx in current_combination], axis=0) # Compute the product of the selected features of list 
            poly_features.append(new_feature)
            return
        
        
        start = current_combination[-1] if current_combination else 0   # Start the loop from the last index used to avoid duplicate combinations
        for i in range(start, n_features):
            recursive_terms(current_degree - 1, current_combination + [i])

    
    for d in range(1, degree + 1):  # Generating terms for each degree from 1 to the given degree
        recursive_terms(d, [])
    
    
    print(poly_features)
    return np.column_stack(poly_features) # Combine all generated features into a single array




In [5]:

degree = 2
x_train = polynomial_features(x, degree = 4)
print(type(x_train))
x_train = (x_train - np.mean(x_train))/(np.max(x_train) - np.min(x_train))
print(x_train)
print(x_train.shape)
print(y_train.shape)


[array([1., 1., 1., ..., 1., 1., 1.]), array([-0.29915022, -0.37431942, -1.24042933, ..., -1.67753838,
        1.31334466, -0.99986957]), array([ 0.98038137, -1.17986704, -0.13908581, ...,  1.75827436,
        0.31535804, -1.6574583 ]), array([-1.24478036,  0.61558815, -0.00165302, ...,  1.02053447,
       -0.06062913, -0.81263227]), array([0.42055144, 0.05470112, 0.4876851 , ..., 0.34649567, 0.70325123,
       1.01871706]), array([-0.35223452,  0.68718223, -0.24280805, ...,  1.31375468,
       -0.61851965, -1.23699276]), array([0.08949085, 0.14011503, 1.53866493, ..., 2.81413501, 1.7248742 ,
       0.99973917]), array([-0.2932813 ,  0.44164715,  0.17252612, ..., -2.94957271,
        0.4141738 ,  1.65724212]), array([ 0.37237631, -0.2304266 ,  0.00205046, ..., -1.71198574,
       -0.07962695,  0.81252628]), array([-0.12580806, -0.02047569, -0.60493891, ..., -0.58125979,
        0.92361125, -1.01858419]), array([ 0.10537103, -0.25722566,  0.30118623, ..., -2.2038739 ,
       -0.81232949

In [6]:

m = x_train.shape[0]
n = int(0.1*m)
x_test1 = x_train[:n,:]
x_train = x_train[n:,:]
print(x_train.shape)


y_test1 = y_train[:n]
y_train = y_train[n:]
print(np.min(y_train))
print(np.max(y_train))
print(y_train.shape)

(43200, 126)
-1.3247832613596604e-07
1.0595967072230566e-07
(43200,)


In [7]:

class KNN:
   def __init__(self,k):
    self.k = k
   def fit(self,x,y):
    self.x = x
    self.y = y
   def dist(self,x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))
   def predict(self,X): 
    prediction = []
    i = 0
    for x in X:
        distance = [self.dist(x,x_train) for x_train in self.x]
        k_indices = np.argsort(distance)[:self.k]
        knearest =  [self.y[i] for i in(k_indices)]
        i = i+ 1
        print(i)
        
        avg = np.mean(knearest)
        prediction.append(avg)
            
            
        
    return np.array(prediction)       

In [ ]:
k = 5
knn = KNN(k)
knn.fit(x_train, y_train)

# Predict
y_pred = knn.predict(x_test1)
print("Predictions:", y_pred)  
print(y_test1)
rmse = np.sqrt(np.mean(y_pred - y_test1)**2)
print(rmse)


ssr = np.sum((y_pred-y_test1)**2)
#print(ssr)
sst = np.sum((y_test1-np.mean(y_test1))**2)
#print(sst)
r2 = 1 - (ssr/sst)
print("R2 SCORE OF MODE ON CROSS VALIDATION IS :",r2)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
data_test = pd.read_csv("polynomial_regression_test.csv")
x_test = data_test.iloc[:,1:]
x_test = (x_test - x_test.mean())/(x_test.std())
x_test = x_test.values
k = 5
knn.fit(x_train, y_train)

# Predict
y_pred = knn.predict(x_test)
print("Predictions:", y_pred)  
print(y_test1)